## Train "by hand" per DPK notebook3 (w/DPK TG, APT dflt IA)

Freeze all artifacts/inputs immediately prior to model.fit

In [1]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import glob
import importlib

import deepposekit as dpk

import TrainingGeneratorTFRecord as TGTFR
import apt_dpk 
import run_apt_expts_2 as rae
import APT_interface as apt
import PoseTools as pt
import multiResData as mrd
import tfdatagen

import time
from os.path import expanduser

import mpl_toolkits.axes_grid1 as axg1

DPK_DSET = '/home/al/git/dpkd/datasets/fly/annotation_data_release.h5'

Using TensorFlow backend.
W0528 21:23:00.239159 140534767683392 deprecation_wrapper.py:119] From /home/al/git/APT_aldl/deepnet/open_pose4.py:12: The name tf.keras.initializers.random_normal is deprecated. Please use tf.compat.v1.keras.initializers.random_normal instead.



Your cache is: /dat0/apt/cache0426
Your models are: ['dpk']


In [2]:
# Create a TGTFR with our default imgaug
LEAPSTRIPPEDLBL = '/dat0/jrcmirror/groups/branson/bransonlab/apt/experiments/data/leap_dataset_gt_stripped.lbl'
EXPNAME = 'val10pct'
CACHE = '/dat0/apt/cache'
RNGSEED = 17

dg = dpk.DataGenerator(DPK_DSET)

conf = apt.create_conf(LEAPSTRIPPEDLBL, 0, EXPNAME, \
                       CACHE, 'dpkfly', quiet=False)
conf.img_dim = 1  # hack, the leap stripped lbl has NumChans=3, but we created the tfr 
                  # directly using the dpk h5 which is 1-channel
conf = apt_dpk.update_conf_dpk(conf,
                               dg.graph,
                               dg.swap_index,
                               n_keypoints=dg.n_keypoints,
                               imshape=dg.compute_image_shape(),
                               useimgaug=True,
                               imgaugtype='dpkfly')
tgtfr = TGTFR.TrainingGeneratorTFRecord(conf)


/dat0/venv/env/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
W0528 21:23:01.515201 140534767683392 deprecation.py:323] From /home/al/git/APT_aldl/deepnet/PoseTools.py:913: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [ ]:
tgtfr.conf.dpk_augmenter.reseed(RNGSEED)
tgtfr.use_tfdata = False
gtf = tgtfr(n_outputs=1, batch_size=8, 
       validation=False, confidence=True, shuffle=False)

imstgts_apt = [next(gtf) for _ in range(3)]

imsapt, tgtsapt = tfdatagen.xylist2xyarr(imstgts_apt, xisscalarlist=True)
imsapt.shape, tgtsapt.shape

In [3]:
tgtfr.conf.dpk_augmenter.reseed(RNGSEED)
tgtfr.use_tfdata = True

In [ ]:
ds = tgtfr(n_outputs=2, batch_size=8, 
       validation=False, confidence=True, shuffle=False)

imstgts_ds = tfdatagen.read_ds_idxed(ds, range(3))

imsDS, tgtsDS = tfdatagen.xylist2xyarr(imstgts_ds)
imsDS.shape, tgtsDS.shape

In [ ]:
np.allclose(imsDS,imsapt), np.allclose(tgtsDS,tgtsapt), \
np.array_equal(imsDS,imsapt), np.array_equal(tgtsDS,tgtsapt), \

In [ ]:
'''
dg = dpk.DataGenerator(DPK_DSET)
ia = apt_dpk.make_imgaug_augmenter('dpkfly', dg)
DSFAC = 2
SIGMA = 5
VALSPLIT = 0.0
GRAPHSCALE = 1
tg = dpk.TrainingGenerator(generator=dg,
                           downsample_factor=DSFAC,
                           augmenter=ia,
                           use_graph=True,
                           shuffle=True,
                           sigma=SIGMA,
                           validation_split=VALSPLIT,
                           graph_scale=GRAPHSCALE,
                           random_seed=0)
'''

In [4]:
tgtfr.use_tfdata

True

In [5]:
sdn = dpk.models.StackedDenseNet(tgtfr, 
                                 n_stacks=2, 
                                 growth_rate=48, 
                                 pretrained=True)


W0528 21:23:49.099966 140534767683392 deprecation.py:506] From /dat0/venv/env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0528 21:23:59.782961 140534767683392 deprecation.py:323] From /dat0/venv/env/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:4075: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


#### Callbacks

In [7]:
conf.cachedir

'/dat0/apt/cache/leap_dset/dpkfly/view_0/val10pct'

In [8]:
cbk_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                              factor=0.2, 
                                              verbose=1, 
                                              patience=20)

cbk_mdlcpt = tf.keras.callbacks.ModelCheckpoint(
    conf.cachedir + "/best_model_sdn.h5",
    monitor="val_loss",  # monitor="loss" # use if validation_split=0
    verbose=1,
    save_best_only=True,
)

#early_stop = EarlyStopping(
#    monitor="val_loss",
#    # monitor="loss" # use if validation_split=0
#    min_delta=0.001,
#    patience=100,
#    verbose=1
#)

callbacks = [cbk_lr, cbk_mdlcpt]

In [ ]:
?tf.keras.Model.fit_generator


In [ ]:
?tf.keras.Model.fit

In [9]:
BSIZE = 8
DLSTEPS = 40000
STEPSPEREPOCH = 50
EPOCHS = DLSTEPS//STEPSPEREPOCH
VALSTEPS = 0  # we aren't using val for anything

In [10]:
EPOCHS

800

In [11]:
sdn.n_outputs

3

In [12]:
dstrn = sdn.train_generator(sdn.n_outputs,8,
                            validation=False,
                            confidence=True,
                            shuffle=True,
                            infinite=True)
                            


W0528 21:31:46.701739 140534767683392 TrainingGeneratorTFRecord.py:255] Ignoring batch specification of 8, conf batchsize is 8
W0528 21:31:46.729755 140534767683392 deprecation_wrapper.py:119] From /home/al/git/APT_aldl/deepnet/tfdatagen.py:666: The name tf.parse_single_example is deprecated. Please use tf.io.parse_single_example instead.

W0528 21:31:46.730829 140534767683392 deprecation_wrapper.py:119] From /home/al/git/APT_aldl/deepnet/tfdatagen.py:668: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.

W0528 21:31:46.755593 140534767683392 deprecation.py:323] From /home/al/git/APT_aldl/deepnet/tfdatagen.py:733: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to con

In [ ]:
resTrn=tfdatagen.read_ds_idxed(dstrn,range(3))


In [13]:
dsval = sdn.train_generator(sdn.n_outputs,8,
                            validation=True,
                            confidence=True,
                            shuffle=False,
                            infinite=False)

W0528 21:32:02.426002 140534767683392 TrainingGeneratorTFRecord.py:255] Ignoring batch specification of 8, conf batchsize is 8


In [ ]:
'''
sdn.fit(
    batch_size=BSIZE, #    validation_batch_size=BSIZE,
    callbacks=callbacks,
    epochs=EPOCHS,
    steps_per_epoch=STEPSPEREPOCH, #    validation_steps=VALSTEPS,
)
'''

In [14]:
sdn.activate_callbacks(callbacks)

In [15]:
m=sdn.train_model

In [16]:
m.compile('adam','mse')

In [17]:
m.fit(dstrn,
      epochs=10,
      steps_per_epoch=5,
      verbose=1,
      callbacks=callbacks,
      validation_data=dsval,
      validation_steps=int(np.ceil(150/8)) )


Epoch 1/10
4/5 [=======================>......] - ETA: 3s - loss: 607.9573 - output_0_loss: 210.4793 - output_1_loss: 202.2973 - output_2_loss: 195.1808
Epoch 00001: val_loss improved from inf to 555.67422, saving model to /dat0/apt/cache/leap_dset/dpkfly/view_0/val10pct/best_model_sdn.h5
5/5 [==============================] - 55s 11s/step - loss: 598.1797 - output_0_loss: 209.7921 - output_1_loss: 199.1656 - output_2_loss: 189.2220 - val_loss: 555.6742 - val_output_0_loss: 206.0405 - val_output_1_loss: 187.4090 - val_output_2_loss: 162.2247
Epoch 2/10
4/5 [=======================>......] - ETA: 2s - loss: 510.0365 - output_0_loss: 201.7024 - output_1_loss: 167.8675 - output_2_loss: 140.4666
Epoch 00002: val_loss improved from 555.67422 to 517.61894, saving model to /dat0/apt/cache/leap_dset/dpkfly/view_0/val10pct/best_model_sdn.h5
5/5 [==============================] - 33s 7s/step - loss: 507.0612 - output_0_loss: 201.7843 - output_1_loss: 165.1151 - output_2_loss: 140.1619 - val_loss

In [ ]:
1350/8

#### Train with APT using dflt IA. 

Freeze all artifacts/inputs. Compare results

#### Train with APT using PT and Bub aug params.
Freeze all artifacts/inputs. Compare results.
(TODO: add this to data_pipeline nb)